In [1]:
def formal_integral(f):
    #Compute the formal integral of a polynomial
    deg = f.degree()
    P = f.parent()
    p = P.characteristic()
    res = 0
    for d in range(0, deg+1):
        if ((d+1) % p != 0):
            res += list(f.coefficients(sparse=False))[d] / (d+1)*P.gen()^(d+1)
    return res

In [2]:
def IntToWitt(k, p, n):
    ZZp = Zp(p, prec=n+1, type = 'fixed-mod')
    F = GF(p)
    series = ZZp(k)
    v = []
    for _ in range(n+1):
        tmp = F(series)
        v.append(tmp)
        
        series = (series - ZZp.teichmuller(tmp))
        if series == ZZp(0):
            continue #Sage has an issue dividing 0 // p?
        else:
            series = series // p
        
        # series = (series - ZZp.teichmuller(tmp)) // p
        
    return v

In [3]:
def etapols(p,n):
    PP.<x,y> = PolynomialRing(Integers(p))
    
    res = []
    for i in range(1, n+1):
        P1.<x,y> = PolynomialRing(Integers(p^(i+1)))
        res.append(x^(p^i) + y^(p^i) - (x+y)^(p^i))
    
    Q.<x,y> = QQ[]
    res[0] = PP(Q(res[0]) / p) #Coerce into an integral domain to do coefficient division
    
    for i in range(2,n+1):
        t = res[i-2]
        
        for j in range(i, n+1):
            P1 = res[j-1].parent()
            t = P1(t)^p
            res[j-1] -= p^(i-1)*t
        
        res[i-1] = PP(Q(res[i-1]) / p^i)
    
    return [x for x in res]

In [4]:
def vRemoveZeros(v=[]):
    tmp = [ x for x in v if x != 0]
    if len(tmp) != 0:
        return tmp
    elif len(tmp) == 0:
        return [0]
    else: #I'm not sure if this case is needed in SAGE
        return [ v[0].parent()(0) ]

In [5]:
def vetav(p, k, v=[], pols=[],check = False):
    # print("v = ", v)
    v = vRemoveZeros(v) 
    lgt = len(v)
    
    if lgt == 1:
        P = v[0].parent()
        return [ P(0) for i in range(0,k) ]
    
    if pols == []:
        pols = etapols(p,k)
        
    #Problem could be here with the evaluation process...maybe
    if lgt == 2: #Note: only evaluates if the number of inputs is the same as the number of indeterminates in pols
        # print("v, pol = ", v, pols)
        return [ pol(v) for pol in pols ]
    else:
        v1 = [ v[i] for i in range(0, floor(lgt / 2))] 
        v2 = [ v[i] for i in range(floor(lgt / 2), lgt)]
        
        x1 = vetav(p, k, v1, pols=pols)
        
        x2 = vetav(p, k, v2, pols=pols, check=True)
        
        x3 = vetav(p, k, [sum(x for x in v1), sum(x for x in v2)], pols=pols)
        
        res = [vRemoveZeros([x1[i], x2[i], x3[i]]) for i in range(0, k) ]
        
        for i in range(2, k+1):
            pols = [ pols[i] for i in range(0, k-i+1)]
            tmp = vetav(p, k-i+1, res[i-2], pols=pols)#Check these indices if something goes wrong
            # print("tmp = ", tmp)
            for t in range(i, k+1):
                if tmp[t-i] != 0:
                    res[t-1].append(tmp[t-i])
        return [ sum(x for x in res[i]) for i in range(0, len(res))]

In [6]:
def WittProd1(v1, v2, pols=[]):
    P = v1[0].parent()
    p = P.characteristic()
    n = len(v1) - 1
    res = [ vRemoveZeros([ v1[j]^(p^(i-j))*v2[i-j]^(p^j) for j in range(0, i+1)]) for i in range(0,n + 1)]
    # print(res) Good to here
    if n == 1:
        return [ sum(x for x in res[i]) for i in range(0, len(res))]
    if pols == []:
        pols = etapols(p, n-1)
    if len(pols) > (n-1):
        pols = pols[0:n]
    # print(pols) Good to here
    
    for i in range(2, n+1):
        l = len(res[i-1])
        for j in range(1, l):
            temp = vetav(p, n+1-i, [res[i-1][j], sum(x for x in res[i-1][0:j])])
            for t in range(i, n+1):
                if temp[t-i] != 0:
                    res[t].append(temp[t-i])

        del pols[n-i] #Check this index if something goes wrong
    
    return [ sum(x for x in res[i]) for i in range(0, len(res))]

In [7]:
def bin_tab(n, m):
    F = Integers(m)
    res = [ [F.one()], [F.one(), F.one()]]
    for i in range(1, n):
        v = [F.one()]
        for j in range(0, i):
            v.append(res[i][j] + res[i][j+1])
        v.append(F.one())
        res.append(v)
    return res

In [8]:
def Not_Zero_Vec(v):
    for i in range(0, len(v)):
        if v[i] != 0:
            return True
        else:
            return False

In [9]:
def GT_der(f, p, i, r, n, tab, pols):
    if r == 0:
        derf = f
    else:
        derf = []
        for t in f:
            coef = t[0]
            degx = t[1]
            degy = t[2]
            
            if (degx - i >= 0) and (degy - (r-i) >= 0) and Not_Zero_Vec(coef):
                bin1 = tab[degx][i]
                bin2 = tab[degy][r-i]
                
                b12 = Integer(bin1*bin2)
                
                if (b12 % p^(n-r+1) == 1):
                    derf.append([ [coef[i] for i in range(0, n+1-r)], degx-i, degy - (r-i)])
                elif (b12 % p^(n-r+1) != 0):
                    v1 = IntToWitt(b12, p, n-r)
                    
                    derf.append([ WittProd1(v1, [ coef[i] for i in range(0, n+1-r)], pols = pols), degx-i, degy - (r-i)])
                    
        if len(derf) == 0:
            derf = [ [ [0 for s in range(0, n-r + 1)], 0, 0 ] ]
        
    return derf

In [10]:
def split_Ds(f, p, upper_bound, PR):
    #print("f, p, upper_bound, PR: ", f, p, upper_bound, PR)
    #print("f exponents: ", f.exponents())
    res = [ PR(0) for i in range(0, upper_bound + 1)]
    N = len(PR.gens())
    for i in range(0, len(f.coefficients())):
        coef = Integer(f.coefficients()[i])
        vcoef = IntToWitt(coef, p, upper_bound)
        temp_exp = PR.one()
        
        #This odd indexing is because Magma assigns generators in the order
        #of x0, x1, x2, y0, y1, y2, while Sage does x0, y0, x1, y1, x2, y2,
        #so in order for the right generators to be picked this must be done
        for k in range(0,2):
            for j in range(0,N // 2):
                temp_exp *= PR.gen(2*j + k)^(f.exponents()[i][(N/2)*k+j]) #Keep an eye on N/2 for correct indexing
        #temp_exp = prod(PR.gen(k)^(f.exponents()[i][k]) for k in range(0,N))
        #print("temp_exp: ", temp_exp)
        for i in range(0, upper_bound + 1):
            res[i] += PR(vcoef[i])*temp_exp
    #print("res in split_Ds: ", res)
    return res

In [11]:
def old_GT_Ds(i, r, n, p, PR):
    PR1 = PolynomialRing(Integers(p^(n+1)), 2*n+2, "alpha00")
    
    PR2.<t> = PolynomialRing(PR1)
    
    tmp = min(n, n-r+1)
    
    t1 = sum(PR2.0^(s+1)*PR1.gen(s+1)^(p^(n-(s+1))) for s in range(0, tmp))
    t2 = sum(PR2.0^(s+1)*PR1.gen(n+s+2)^(p^(n-(s+1))) for s in range(0, tmp))

    t = t1^i*t2^(r-i)
    #print(t)
    
    t = t.coefficients(sparse=False)[r:n+1]
    #print(t)
    
    return [ split_Ds(f, p, n-r, PR) for f in t]

In [12]:
def GT_Ds(i, r, n, p, PR):
    PR1 = PolynomialRing(Integers(p^(n+1)), 2*n+2, "alpha00")
    
    PR2.<t> = PolynomialRing(PR1)
    
    tmp = min(n, n-r+1)
    
    t1 = sum(PR2.0^(s+1)*PR1.gen(s+1)^(p^(n-(s+1))) for s in range(0, tmp))
    t2 = sum(PR2.0^(s+1)*PR1.gen(n+s+2)^(p^(n-(s+1))) for s in range(0, tmp))
    #print ("t1 = ", t1)
    #print ("t2 = ", t2)
    t = t1^i*t2^(r-i)
    
    #Deal with the case that the degree of t is less than n+1, so the
    #coefficients on higher powers get grabbed (though they are 0)
    if t.degree() < n+1:
        s = []
        for i in range(r, t.degree()+1):
            s.append(t.coefficients(sparse=False)[i])
        for i in range(t.degree() + 1, n+1):
            s.append(PR2(0))
    else:
        s = t.coefficients(sparse=False)[r:n+1]
    
    #print(s)
    #print(" s[0], p, n-r, PR = ", s[0], p, n-r, PR)
    return [ split_Ds(f, p, n-r, PR) for f in s]

In [13]:
def Pol_Root(f, r):
    P = f.parent()
    n = len(P.gens())
    res = 0
    for i in range(0, len(f.coefficients())):
        nmon = f.coefficients()[i]
        for j in range(0,n):
            nmon *= (P.gen(j))^(f.exponents()[i][j] // r)
        res += nmon
    return res

In [14]:
def Permute_Entries(array=[]):
    permute = []
    k = len(array) // 2
    for i in range(0, len(array) // 2):
        permute.append(array[i])
        permute.append(array[k+i])
    return permute

In [15]:
def GT1(f, pols=[], tab=[], vvars=[]):
    n = len(f[0][0]) - 1
    P = f[0][0][0].parent()
    p = P.characteristic()
    
    if len(pols) == 0:
        pols = etapols(p,n)
    
    PR = PolynomialRing(P, n+1, var_array=['x','y'])
    
    maxdegx = max(t[1] for t in f)
    maxdegy = max(t[2] for t in f)
    maxdeg = max(maxdegx, maxdegy)
    
    if len(tab) == 0:
        tab = bin_tab(maxdeg, p^(n+1))
        
    res = [ [] for s in range(0, n+1) ]
    
    for r in range(0, n+1):
        for i in range(0, r+1):
            # print("r, i = ", r, i)
            if (i <= maxdegx) and (r-i <= maxdegy):
                #print("f, p, i, r, n, tab, pols =", f, p, i, r, n, tab, pols)
                derf = GT_der(f, p, i, r, n, tab, pols)
                #print("derf = ", derf)
                
                #print("i, r, n, p, PR = ", i, r, n, p, PR)
                Ds = GT_Ds(i, r, n, p, PR)
                # print("r, i = ", r, i)
                
                for m in range(r, n+1):
                    for j in range(r, m+1):
                        for k in range(r, j+1):
                            vt = sum( (t[0][m-j])^(p^j)*(PR.0)^(t[1]*p^m)*(PR.1)^(t[2]*p^m) for t in derf )
                            
                            v = [ PR(0) for s in range(0, 2*n+2) ]
                            for s in range(1, (m+1)):
                                v[2*s] = PR.gen(2*s)
                                v[2*s+1] = PR.gen(2*s+1)
                            
                            #print("r, i, m, j, k =", r, i, m, j, k)
                            #print("Ds = ", Ds)
                            #print("v = ", v)
                            #print("Evaluate at ", k-r, j-k)
                            #if [r, i, m, j, k] == [2, 2, 2, 2, 2]:
                                #return None
                            tmp = Ds[k-r][j-k](v)
                            #print("tmp before =", tmp)
                            tmp = Pol_Root(tmp, p^(n-m))
                            #print("tmp after: ", tmp)
                            #print("vt: ", vt)
                            prdct = vt*tmp
                            #print("vt = ", vt)
                            #print("tmp = ", tmp)
                            #print("prdct = ", prdct)
                            #print("vvars = ", vvars)
                            #Problem is here, try checking that it's adding things properly to res
                            if len(vvars) != 0:
                                for ii in range(0, len(list(prdct))):
                                    new_vvars = Permute_Entries(vvars) #Get in form (x0, y0, x1, y1, ...)
                                    #print("new_vvars: ", new_vvars)
                                    res[m] += [list(prdct)[ii][0]*(list(prdct)[ii][1](new_vvars))]
                            else:
                                for ii in range(0, len(list(prdct))):
                                    #print("Adding to res[{}]: ".format(m), [list(prdct)[ii][0]*(list(prdct)[ii][1])])
                                    res[m] += [list(prdct)[ii][0]*(list(prdct)[ii][1])]
                            # print("r, i, m, j, k =", r, i, m, j, k)
                            # print("res = ", res)

    #print("res = ", res)
    #print("memory usage = ", get_memory_usage())
    for i in range(0, n):
        # print("p, n-i, res[",i,"], pols = ", p, n-i, res[i], pols)
        ve = vetav(p,n-i, res[i], pols=pols)
        # print("ve = ", ve)
        for j in range((i+1), (n+1)):
            if ve[j-(i+1)] != 0:
                res[j].append(ve[j-(i+1)])
                #print("i, j = ", i, j)
                #print("res = ", res)
        pols = pols[0:(n-(i+1))]
    #print("res = ", res)
    #return None
    return [ sum(t for t in res[i]) for i in range(0,len(res))]

In [16]:
p = 3; d = 4; F.<a>=GF(p^d);
epols = etapols(p,2);
P.<x>=PolynomialRing(F);
v=[a^29, x + a^11, a^68]
w=[a^25, a^48, a^9*x^2]

vone = [F(1), F(0), F(0)]
vtwo = IntToWitt(2, p, 2)

pol = [ [vone, 2, 1], [vtwo, 1, 1], [vone, 0, 3] ]

In [17]:
GT1(pol, pols=epols, vvars= v + w)

[2*a^2 + 2*a + 2,
 (2*a^3 + a^2 + 2*a)*x + 2*a^3 + a^2 + 2,
 (2*a^2 + 1)*x^6 + (a^3 + a^2)*x^3 + (2*a^3 + a + 1)*x^2 + (2*a^2 + a + 2)*x + 1]

In [18]:
p = 3; d = 4; F.<a> = GF(p^d);
pol = [ [ [a, F(2), a^5], 2, 0 ], [ [a, F(0), F(1)], 1, 1 ], [ [F(1), a^7, F(0) ], 0 , 0 ] ]
GT1(pol)

[(a)*x0^2 + (a)*x0*y0 + 1,
 -x0^6 + (-a^3)*x0^5*y0 + (-a^3)*x0^4*y0^2 + (-a^2)*x0^4 + (a^2)*x0^3*y0 + (-a^2)*x0^2*y0^2 + (-a^3)*x0^3*x1 + (a^3)*y0^3*x1 + (a^3)*x0^3*y1 + (-a)*x0^2 + (-a)*x0*y0 + (-a^3 + a^2 + a + 1),
 (a^3 + a + 1)*x0^18 + (-a^2 + a)*x0^17*y0 + (a^3 - a + 1)*x0^16*y0^2 + (-a^3 - a^2 - a - 1)*x0^15*y0^3 + (-a + 1)*x0^14*y0^4 + (-a^3 - a^2 + a)*x0^13*y0^5 + (-a^3 - a^2 + a)*x0^11*y0^7 + (-a^3 - a^2 + a)*x0^10*y0^8 + x0^9*y0^9 + (-a + 1)*x0^16 + (-a^3 + 1)*x0^15*y0 + (a^2)*x0^14*y0^2 + (-a^2 - a + 1)*x0^13*y0^3 + (-a^3 - a^2 + a)*x0^12*y0^4 + (-a^2 - a + 1)*x0^11*y0^5 + (a^2 + a - 1)*x0^9*y0^7 + (-a^2 - a + 1)*x0^8*y0^8 + (a^3)*x0^15*x1 + (-a^3 - a^2 - a - 1)*x0^14*y0*x1 + (a - 1)*x0^13*y0^2*x1 + (a^3 - a^2 + a)*x0^12*y0^3*x1 + (-a^3 - a^2 + 1)*x0^11*y0^4*x1 + (-a + 1)*x0^10*y0^5*x1 + (a^3 + a^2 - a)*x0^9*y0^6*x1 + (-a^3 - a^2 + a)*x0^8*y0^7*x1 + (-a^3)*x0^15*y1 + (a^3 + a^2 + a + 1)*x0^14*y0*y1 + (-a + 1)*x0^13*y0^2*y1 + (a^3 + a^2 - a)*x0^12*y0^3*y1 + (-a^3 - a^2 + a)*x

In [65]:
def lift(a0, b0, n, pols=[], minimal=False):
    # length (n+1) -- up to a_n, b_n
    # returns vectors va, vb, vF, vH
    # pols are etapols from etas.m
    # computes canonical lifting by default
    # set minimal := true to compute minimal degree liftings
    
    F = (a0.parent()).fraction_field()
    p = F.characteristic()
    
    if len(pols) == 0:
        pols = etapols(p,n)
        
    Pres.<x0> = PolynomialRing(F)
    
    resa = [a0]
    resb = [b0]
    resF = [x0]
    resH = [Pres.one()]
    
    f = x0^3 + a0*x0 + b0
    ff = f^((p-1) // 2)
    
    HI = F(ff.coefficients(sparse=False)[p-1]) #Determine the Hasse Invariant
    
    for i in range(0, n):
        
        M = (3*p^(i)-1) // 2
        Pi = PolynomialRing(F, ['x_0','y_0', 'a_n', 'b_n'] + [ 'c_%s'%i for i in range(0,M+1)] + ['Hn'])
        
        if i == 0:
            tmppf = ff
        else:
            tmppf = tmppf^p*ff
        
        Fi = HI^(-(p^(i+1)-1) / (p-1))*tmppf - x0^(p^(i+1)-1)
        #print("Fi: ", Fi)
        if i > 0:
            Fi -= sum( resF[j]^(p^((i+1)-j)-1)*resF[j].derivative() for j in range(1, i+1) )
        #print("Fi: ", Fi)
        
        Fi = formal_integral(Fi)
        Fi = Fi(Pi.gen(0))
        
        Fi += sum( Pi.gen(4+j)*(Pi.0)^(p*j) for j in range(0,M+1) if j != p^(i) )
        #print("Fi: ", Fi)   
            
        if (not minimal) and (i == 1):
            tmp = F(3/4)*resF[1]^2
            Fi += sum( tmp.coefficient({x0:Integer(i/p + p)})^p*(Pi.0)^i for i in range(((3*p^2+p)/ 2), 2*p^2-p + 1, p))
            
        va = [ Pi(x) for x in resa] + [Pi.2]
        vb = [ Pi(x) for x in resb] + [Pi.3]
        vF = [ x(Pi.0) for x in resF] + [Fi]
        vG = [ Pi.1*(x(Pi.0)) for x in resH ] + [Pi.1*Pi.gen(M+5)]
        vone = [Pi.one()] + [Pi.zero() for j in range(0, i + 1)]
        
        #print("va: ", va)
        #print("vb: ", vb)
        #print("vF: ", vF)
        #print("vG: ", vG)
        
        vvars = vF + vG
        
        GTx = GT1( [[ vone, 3, 0], [va, 1, 0], [vb, 0, 0]], pols=pols, vvars=vvars)
        #print("vone: ", vone)
        #print("pols= ", pols)
        #print("vvars= ", vvars)
        GTy = GT1( [[ vone, 0, 2]], pols=pols, vvars=vvars)
        
        RHS = GTx[i+1]
        LHS = GTy[i+1]
        
        #print("GTy= ", GTy)
        #print("LHS= ", LHS)
        #print("GTx= ", GTx)
        #print("RHS= ", RHS)
        #print("LHS Parent: ", LHS.parent())
        LHS = LHS.coefficient({Pi.gen(M+5):0}) #Terms without Hn?
        
        #print("LHS: " , LHS)
        
        deg = LHS.degree(Pi.1)
        tmppf2 = 1
        tmpLHS = LHS.coefficient({Pi.1:0})
        for d in [ j for j in range(2, deg+1) if j % 2 == 0]:
            tmppf2 *= f(Pi.0)
            tmpLHS += LHS.coefficient({Pi.1:d})*tmppf2
            
        RHS -= tmpLHS
        
        tmppf2 = (tmppf*f)(Pi.0)
        
        RHS *= Pi(1/2)
        deg1 = RHS.degree(Pi.0)
        deg2 = (3*(p^(i+1)+1) / 2)
        quo = 0
        #print("RHS start: ", RHS)
        
        while deg1 >= deg2:
            lterm = RHS.coefficient({Pi.0:deg1})*((Pi.0)^(Integer(deg1-deg2)))
            RHS -= lterm*tmppf2
            #print("RHS: ", RHS)
            quo += lterm
            deg1 = RHS.degree(Pi.0)
            
        vrem = []
        for jj in range(0, RHS.degree(Pi.0)+1):
            vrem += [ RHS.coefficient({Pi.0:jj})]
            
        neqts = len(vrem)

        Mat = Matrix(F, [ [F(vrem[j].coefficient({Pi.gen(k+1):1})) for j in range(0, neqts)]
                         for k in  [ ii for ii in range(1, (2+(M+1))+1) if ii != 3+p^(i) ]])
        
        #print("vrem: ", vrem)
        vec = vector(F, [ -vrem[j]([Pi(0) for k in range(1, 2 + 2 + (M+1) + 2)]) for j in range(0, neqts)])

        #print("Mat: ", Mat)
        #print("vec: ", vec)
        
        vsol = Mat.solve_left(vec)
        
        evalvec = [x0, 0] + [vsol[j] for j in range(0, 2+p^(i))] + [0] + [vsol[j] for j in range(2+p^(i), (2+M))] + [0]
        #print("evalvec: ", evalvec)
        
        resa.append(vsol[0])
        resb.append(vsol[1])
        resF.append(Fi(evalvec))
        resH.append(quo(evalvec))
        #print("i= ", i)
        #print("resa: ", resa)
        #print("resb: ", resb)
        #print("resF: ", resF)
        #print("resH: ", resH)
        
    return resa, resb, resF, resH

In [66]:
epols = etapols(5,3)
lift(GF(5)(1),GF(5)(2),3, minimal=True, pols=epols)

([1, 0, 4, 0],
 [2, 3, 3, 0],
 [x0,
  4*x0^7 + 3*x0^4 + x0^3 + x0^2 + 2*x0 + 2,
  2*x0^37 + 4*x0^34 + 3*x0^33 + x0^30 + x0^29 + 3*x0^27 + 4*x0^26 + 2*x0^24 + 2*x0^23 + 4*x0^22 + x0^20 + 4*x0^19 + 2*x0^17 + x0^16 + x0^15 + 2*x0^14 + 4*x0^11 + x0^10 + 4*x0^9 + 4*x0^8 + 4*x0^7 + 2*x0^6 + 3*x0^5 + 3*x0^3 + 2*x0^2 + 2*x0 + 1,
  x0^187 + 3*x0^185 + 2*x0^184 + 4*x0^183 + 3*x0^180 + 4*x0^179 + 2*x0^177 + 3*x0^176 + x0^175 + 2*x0^174 + 4*x0^173 + 3*x0^172 + 4*x0^171 + 3*x0^170 + 2*x0^169 + 3*x0^167 + x0^166 + 2*x0^165 + 2*x0^164 + 3*x0^163 + 2*x0^162 + x0^161 + 3*x0^159 + x0^158 + 2*x0^156 + 3*x0^155 + 2*x0^154 + 2*x0^153 + 2*x0^152 + 3*x0^149 + 4*x0^147 + 3*x0^143 + x0^142 + 2*x0^141 + 4*x0^139 + 4*x0^137 + x0^136 + 3*x0^134 + 2*x0^133 + 3*x0^132 + 4*x0^131 + 2*x0^130 + 4*x0^129 + 2*x0^128 + 2*x0^126 + 2*x0^123 + 4*x0^122 + 4*x0^121 + x0^119 + x0^118 + 4*x0^117 + 4*x0^116 + 3*x0^115 + 3*x0^113 + 2*x0^112 + 3*x0^111 + 3*x0^110 + 3*x0^109 + x0^108 + 2*x0^106 + 3*x0^105 + 3*x0^104 + 2*x0^103 + 4*

In [67]:
lift(a0, b0, 1)

([a0, (a0^3*b0^2 + b0^4)/a0],
 [b0, -a0^6*b0 + a0^3*b0^3 + b0^5],
 [x0,
  (1/(-a0))*x0^7 + (2*b0/(-2*a0))*x0^4 + a0*x0^3 - 2*b0*x0^2 + b0^2/(2*a0)*x0 + a0*b0],
 [1,
  ((-1)/(-a0))*x0^8 + 2*x0^6 + b0/(2*a0)*x0^5 + 2*a0*x0^4 - 2*b0*x0^3 + a0^2*x0^2 - 2*a0*b0*x0 - 2*a0^3 - 2*b0^2])

In [17]:
p = 5; 
R.<a0, b0> = PolynomialRing(GF(p),2)
F = R.fraction_field()
F

Fraction Field of Multivariate Polynomial Ring in a0, b0 over Finite Field of size 5

In [68]:
lift(a0, b0, 2, minimal=True)

([a0,
  (a0^3*b0^2 + b0^4)/a0,
  (a0^36 - a0^33*b0^2 - a0^30*b0^4 + 2*a0^27*b0^6 - 2*a0^24*b0^8 - a0^18*b0^12 + a0^12*b0^16 + 2*a0^9*b0^18 + a0^6*b0^20 + a0^3*b0^22 + b0^24)/(-a0^11)],
 [b0,
  -a0^6*b0 + a0^3*b0^3 + b0^5,
  a0^36*b0 - a0^33*b0^3 - 2*a0^27*b0^7 - a0^21*b0^11 - a0^15*b0^15 + a0^12*b0^17 - 2*a0^6*b0^21 - b0^25],
 [x0,
  (1/(-a0))*x0^7 + (2*b0/(-2*a0))*x0^4 + a0*x0^3 - 2*b0*x0^2 + b0^2/(2*a0)*x0 + a0*b0,
  (1/(-2*a0^6))*x0^37 + ((-2*a0^3*b0^2 - 2*b0^4)/(-2*a0^11))*x0^35 + 2*b0/a0^6*x0^34 + 1/(2*a0^4)*x0^33 + ((-a0^3 - b0^2)/a0^6)*x0^31 + b0^5/(2*a0^10)*x0^30 + ((-b0^2)/a0^5)*x0^29 + ((-a0^3 + 2*b0^2)/(-a0^4))*x0^27 + ((-2*b0)/(-a0^2))*x0^26 + ((-2*a0^3*b0 + b0^3)/(2*a0^4))*x0^24 + ((a0^6 + 2*a0^3*b0^2 - b0^4)/(-a0^5))*x0^23 + ((-2*a0^3*b0^3 + b0^5)/(-a0^6))*x0^22 + ((a0^3*b0^2 + b0^4)/a0^4)*x0^21 + ((a0^12*b0 + 2*a0^9*b0^3 - 2*a0^3*b0^7 - 2*b0^9)/(-2*a0^11))*x0^20 + ((a0^9 - 2*a0^6*b0^2 - b0^6)/a0^6)*x0^19 + ((-2*a0^6*b0 - a0^3*b0^3 + b0^5)/(-a0^4))*x0^18 + ((-a0^6 + 2*a0^

In [49]:
p = 5
i = 0
M = 1
v = [ ii for ii in range(1, (2+(M+1))+1) if ii != 3+p^(i) ]
v

[1, 2, 3]

In [90]:
epols = etapols(5,3)
lift(GF(5)(1),GF(5)(2), 3, minimal=True, pols = epols)

Mat:  [0 0 0 0 0 3 0 0 0]
[3 0 0 0 0 0 0 0 0]
[3 3 2 1 3 2 3 1 3]
vec:  (0, 1, 4, 2, 1, 4, 1, 2, 1)
Mat:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0]
[3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[3 4 1 3 4 4 3 4 2 4 1 0 2 1 1 3 0 3 4 4 1 1 1 3 2 3 2 4 2 0 2 1 4 2 1 0 1 2 1]
[1 3 2 1 3 4 1 3 4 3 3 0 4 2 2 2 0 1 3 3 1 2 2 1 4 1 4 3 4 0 2 2 3 4 2 2 2 4 2]
[2 2 3 4 2 2 4 2 1 2 3 0 1 3 3 3 0 4 2 2 0 3 3 4 1 0 1 2 1 0 0 3 2 1 3 3 3 1 3]
[3 3 2 1 3 1 1 3 4 3 3 0 4 2 2 3 0 1 3 3 0 2 2 1 4 1 4 3 4 0 1 2 3 4 2 4 2 4 2]
[2 3 2 1 3 3 1 3 4 3 1 0 4 2 2 1 0 1 3 3 3 2 2 1 4 3 4 3 4 0 1 2 3 4 2 3 2 4 2]
[0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0]
vec:  (0, 3, 2, 1, 4, 4, 2, 2, 2, 1, 4, 4, 4, 0, 1, 0, 1, 0, 3, 3, 2, 2, 3, 2, 4, 3, 3, 2, 3, 3, 0, 1, 3, 1, 1, 3, 4, 0, 0)


ValueError: matrix equation has no solutions

In [19]:
F.<x,y> = PolynomialRing(GF(5),2)
f = 2*x + 3*x*y

In [64]:
f.coefficient({x:1})

-2*y + 2

In [83]:
G.<x0> = GF(5)[]
g = 1*x0 - 2*x0^2 + 3*x0^3

In [92]:
v = [i for i in range(0,0)]
v

[]

In [23]:
p = 5; d = 3; F.<a> = GF(p^d); P.<x,y> = F[]
v = [x, x*y, P(2)]
w = [x^2, x-y, 1 + y^2]

In [27]:
epols = etapols(p,2)
WittProd1(v, w, pols=epols)

[x^3, x^11*y + x^6 - x^5*y, 2*x^50 + x^25*y^2 + x^25 + x^10*y^5 - x^5*y^10]

In [15]:
f = 3*x0^4 + 2*x0^2 - x0 + 7
print(f.coefficients(sparse=False))
print(list(f))
print(f)

NameError: name 'x0' is not defined

In [17]:
bin_tab(3,4)

[[1], [1, 1], [1, 1], [1, 2, 1], [1, 3, 3, 1]]

In [58]:
F = PolynomialRing(GF(7), ['x_n','y_n', 'a_n', 'b_n'] + [ 'c%s'%i for i in range(0,5+1)] + ['Hn'])
f = F.1^7 + F.2*F.0^6
list(f)
G = PolynomialRing(GF(7), 'x0')
g = x0^2 + 3*x0
G.gen()(5)

5

In [83]:
P = PolynomialRing(GF(7), 3,  var_array = ['x'])
g = P.0^3 * P.1 - 5 * P.2 * P.1^2 + 5
test = []
for i in range(0, g.degree(P.0)+1):
    test += [ g.coefficient({P.0:i})]
test

[2*x1^2*x2 - 2, 0, 0, x1]

In [73]:
f.coefficient(F.0^0)

y_n^7 + x_n^6*a_n

In [85]:
M = Matrix(ZZ, [ [i + j for i in range(0,5) ] for j in range(0,5)]); M

[0 1 2 3 4]
[1 2 3 4 5]
[2 3 4 5 6]
[3 4 5 6 7]
[4 5 6 7 8]

In [86]:
M[2][2].parent()

Integer Ring

In [51]:
p = 5
n = 6
PR1 = PolynomialRing(Integers(p^(n+1)), 2*n+2, "alpha00")
PR2.<t> = PolynomialRing(PR1)
t1 = sum(PR2.0^(s+1)*PR1.gen(s+1)^(p^(n-(s+1))) for s in range(0, n))
t2 = sum(PR2.0^(s+1)*PR1.gen(n+s+2)^(p^(n-(s+1))) for s in range(0, n))

In [130]:
res = [ [] for s in range (0, 3) ]
res

[[], [], []]

In [149]:
v = [1, 2, 3, 4, 5]
h(v)

63

In [136]:
F = GF(7)
PR = PolynomialRing(F, 5, var_array = ['x', 'y'])
for i in range(0, 10):
    print(PR.gen(i))

x0
y0
x1
y1
x2
y2
x3
y3
x4
y4


In [140]:
v = [ PR(0) for s in range(0, 2*4+2)]
for s in range(1, 5):
    v[s] = PR.gen(2*s)
    v[s+4+1] = PR.gen(2*s+1)
v

[0, x1, x2, x3, x4, 0, y1, y2, y3, y4]

In [1]:
F = PolynomialRing(GF(7), 4, var_array = ['x','y'])
f = F.1^4*F.0^2 + F.4*5 - F.5^2 * F.7^3*10 - 15
f

x0^2*y0^4 - 3*y2^2*y3^3 - 2*x2 - 1

In [2]:
list(f)

[(1, x0^2*y0^4), (4, y2^2*y3^3), (5, x2), (6, 1)]

In [5]:
for i in range(0, len(list(f))):
    print(list(f)[i][1]*list(f)[i][0])

x0^2*y0^4
-3*y2^2*y3^3
-2*x2
-1


In [21]:
def my_SeriesToWitt(s, F=None):
    ZZq = s.parent()
    n = len(list(b.expansion()))
    k = ZZq.degree()
    p = ZZq.prime()
    a0 = ZZq.gen()
    
    if F is None:
        F.<a> = GF(p^k)
    
    FFq = ZZq.residue_field()
    b0 = FFq.gen()
    phi = F.hom([b0]) #Map from F to FFq
    
    v = []
    tmp_s = s
    
    for i in range(0, len(list(s.expansion()))):
        tmp = F(0)
        for j in range(len(list(tmp_s.expansion()[0]))):
            tmp += (F(tmp_s[i][j])*F.gen()^j)*(p^i) #Coerce s_i into F
        v.append(tmp)
        tmp_lift = ZZq(ZZq.teichmuller(phi(tmp)))
        tmp_s = (tmp_s - tmp_lift) / p
    
    return [ v[i]^(p^i) for i in range(len(v)) ]

In [2]:
def WittToSeries(v, ZZq = None):
    FF = v[0].parent()  # base field
    p = FF.characteristic()
    degree = FF.degree()
    a = FF.gen()
    n = len(v) - 1
    
    # if ring is not given, create it
    if ZZq is None:
        ZZq.<A> = Zq(p^degree, n+1)
        
    FFq = ZZq.residue_field()
    a0 = FFq.gen()
    
    # We now need a map from FF and FFq
    phi = FF.hom([a0])
    
    return sum([ ZZq.teichmuller(phi(x).pth_root(i)) * p^i for i, x in enumerate(v) ])